In [1]:
from zumi.zumi import Zumi
from zumi.util.screen import Screen
from zumi.protocol import Note
from zumi.util.camera import Camera
from zumi.util.color_classifier import ColorClassifier
from threading import Thread
from zumi.util.vision import Vision
from zumi.util.gyro_draw import Gyro_Draw
import IPython.display
import numpy
import PIL.Image
import time

zumi = Zumi()
screen = Screen()
camera = Camera()
vision = Vision()
now_angle = zumi.read_z_angle()

user_name = 'User1'
demo_name = 'color'
knn = ColorClassifier(user_name=user_name)
train = knn.load_model(demo_name)
knn.fit("hsv")

Starting Zumi 
Pi Zero I2C is available
Verified Pi Zero is the same
Gyroscope previously calibrated
Zumi board detected
OLED Screen detected
Gyroscope & Accelerometer detected
Zumi battery level is low  22 %
Please charge me!
Loaded  color  successfully


In [2]:
threshold=150
turnSpeed=8
forwardSpeed=8

def bottom_read() :
    global bottom_right, bottom_left
    bottom_right = zumi.read_IR('bottom_right')
    bottom_left = zumi.read_IR('bottom_left')
    
def qrcode():
    global camera, vision
    count = 0
    while True:
        camera.start_camera()
        image = camera.capture()

        qr_code = vision.find_QR_code(image)
        qr_message = vision.get_QR_message(qr_code)
        value = str(qr_message)

        if(qr_message == None):
            continue
        else:
            if(value == 'factory'):
                screen.draw_text_center(str(value))
                count = 3
                direct = "right"
                return count, direct
            
            elif(value == "school"):
                screen.draw_text_center(str(value))
                count = 7
                direct = "left"
                return count, direct
                
            elif(value == "building"):
                screen.draw_text_center(str(value))
                count = 11
                direct = "right"
                return count, direct
                
            elif(value == "museum"):
                screen.draw_text_center(str(value))
                count = 15
                direct = "left"
                return count, direct
                    
            elif(value == "left"):
                screen.draw_text_center(str(value))
                direct = value
                return count, direct

            elif(value == "right"):
                screen.draw_text_center(str(value))
                direct = value
                return count, direct
            
def sensor():
    try:
        for i in range(100):
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')
            if front_left > 100 and front_right > 100:
                zumi.stop()

            elif front_left > 100:
                zumi.turn_right()

            elif front_right > 100:
                zumi.turn_left()

            else:
                zumi.forward()
        

    except KeyboardInterrupt:
            zumi.stop()

def enter_lane_trace() : 
    global bottom_right, bottom_left
    bottom_read()
    
    turnSpeed = 3
    forwardSpeed = 1

    if bottom_left < threshold and bottom_right > threshold :
        zumi.stop()
        zumi.control_motors(0,turnSpeed,0)
    elif bottom_left > threshold and bottom_right < threshold :
        zumi.stop()
        zumi.control_motors(turnSpeed + 5,0,0)
    else :
        zumi.control_motors(forwardSpeed+4,forwardSpeed,0)
        
def enter_course() :
    check = 0
    while True:
        enter_lane_trace()
        if isWhite():
            check += 1
            zumi.reverse(15, 0.3)
            time.sleep(0.5)

        if check > 2:
            break
            
def lane_trace() :
    global bottom_right, bottom_left
    bottom_read()

    if bottom_left < threshold and bottom_right > threshold :
        zumi.stop()
        zumi.control_motors(0,turnSpeed,0)
    elif bottom_left > threshold and bottom_right < threshold :
        zumi.stop()
        zumi.control_motors(turnSpeed,0,0)
    else :
        zumi.control_motors(forwardSpeed,forwardSpeed // 2,0)
        
def isWhite() :
    if bottom_right < threshold and bottom_left < threshold :
        return True
    else :
        return False

In [3]:
# A course

def acourse():
    zumi.reset_gyro()
    count, direct = qrcode()
    zumi.drive_over_markers(count, 10, 125, 10)
    zumi.stop()
    if direct == "right":
        zumi.turn_right(90)
    elif direct == "left":
        zumi.turn_left(90)
    zumi.forward(duration = 1.5)
    time.sleep(2)
    if direct == "right":
        zumi.turn(90)
        zumi.forward(duration=1.5)
        zumi.turn_right(90)
        zumi.turn_left(-now_angle)
    elif direct == "left":
        zumi.turn(-90)
        zumi.forward(duration=1.5)
        zumi.turn_left(90)
        zumi.turn_right(-now_angle)
        
    zumi.forward_avoid_collision (45, 15, None, 65, 65)
    
    zumi.turn_right(90,1)
    
    camera.close()

    zumi.control_motors(20,10,0)
    while True:
        front_right = zumi.read_IR('front_right')
        front_left = zumi.read_IR('front_left')
        
        if front_left < 100 or front_right < 100:
            zumi.turn(-80)
            break
    enter_course()

In [4]:
#B course

def bcourse() :
    
    camera.start_camera()
    image = camera.capture()
    predict = knn.predict(image)
    zumi.stop()
    screen.draw_text_center(predict)
    camera.close()
    
    try:
        zumi.reset_gyro()
        
    
    except KeyboardInterrupt:
        zumi.stop()
    
    enter_course()
    camera.close()


In [5]:
#C course

def ccourse():
    try:
        zumi.forward(speed=20, duration=0.5)
        check = 0
        
        while True:
            lane_trace()
            if isWhite():
                zumi.reverse(10, 0.1)
                time.sleep(0.5)
            
                if check == 0:
                    zumi.turn_left(78)
                elif check == 1 or check == 2:
                    zumi.turn_right(78)
                else:
                    break
                
                check += 1
                
        value = ""
        camera.start_camera()
        while True:
            image = camera.capture()

            qr_code = vision.find_QR_code(image)
            qr_message = vision.get_QR_message(qr_code)

            value = qr_message

            if(qr_message == None):
                continue
            else:
                break

        check = 0

        if value == 'right':
            # right
            zumi.turn_right(50)
        elif value == 'left':
            # left
            zumi.turn_left(50)
            
        while True:
            zumi.reverse(15, 0.05)
            
            bottom_right = zumi.boolean_IR('bottom_right', 130)
            bottom_left = zumi.boolean_IR('bottom_left', 130)   
            if not bottom_right or not bottom_left:
                break
                
        while True:
            lane_trace()
            if isWhite():
                if value == 0:
                    zumi.turn_left(40)
                else:
                    zumi.turn_right(40)
                break
                
        while True:
            zumi.reverse(15, 0.05)
            
            bottom_right = zumi.boolean_IR('bottom_right', 130)
            bottom_left = zumi.boolean_IR('bottom_left', 130)  
            
            if not bottom_right or not bottom_left:
                break
            
        while True:
            lane_trace()
            if isWhite():
                break
                
        zumi.forward(speed=13, duration=1)
            
                
    except KeyboardInterrupt:
        camera.close()
        zumi.stop()

In [6]:
acourse()



bcourse()


ccourse()

Starting PiCamera
Closing PiCamera


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):



KeyboardInterrupt

